# Zollman Effect

## Running a single simulation

In [ ]:
# Enable auto-reloading
%reload_ext autoreload
# 2: Reload all
%autoreload 2

import polygraphs as pg

from polygraphs import hyperparameters as hparams
from polygraphs import ops


# Create a PolyGraph configuration
params = hparams.PolyGraphHyperParameters()

# Initial beliefs are random uniform between 0 and 1
params.init.kind = 'uniform'
# Chance that action B is better than action A
params.epsilon = 0.01
# Kind and size of network (a complete network with 10 agents)
params.network.kind = 'complete'
params.network.size = 16

# Enable logging; print progress every 100 steps
params.logging.enabled = True
params.logging.interval = 100

# Run 1,000 steps per simulation; repeat simulation 10 times
params.simulation.steps = 1000
params.simulation.repeats = 10

# Set seed
params.seed = 123456789

pg.random(params.seed)
_ = pg.simulate(params, op=ops.BalaGoyalOp)

print('Bye.')

## Running experiments

### 1 The basic idea

#### 1.1 The code

In [ ]:
# Enable auto-reloading
%reload_ext autoreload
# 2: Reload all
%autoreload 2


from collections import deque, namedtuple

import polygraphs as pg

from polygraphs import hyperparameters as hparams
from polygraphs import metadata
from polygraphs import ops


# Create base PolyGraph configuration. All simulations inherit from this configuration.
# In this particular case, we will vary the network size.
params = hparams.PolyGraphHyperParameters()

# Initial beliefs are random uniform between 0 and 1
params.init.kind = 'uniform'
# Chance that action B is better than action A
params.epsilon = 0.01
# Kind and size of network (a complete network with N agents)
params.network.kind = 'cycle'
params.network.size = None
# Disable logging
params.logging.enabled = False
# Repeat simulation 10 times; run to convergence
params.simulation.steps = 0
params.simulation.repeats = 10

# Generate a list of configurations, varying the network size
options = {'network.size': [2, 4, 8]}
configurations = hparams.PolyGraphHyperParameters.expand(params, options)

results = deque()
for config in configurations:
    # Run experiment
    pg.log.info('Simulating a {} network of size {}'.format(config.network.kind, config.network.size))
    # Besides config and op, the rest of the keyword arguments (e.g. size=N)
    # are treated as metadata. Metadata helps distinguish simulation results
    # in a collection (e.g. `results`) and, subsequently, post-processing.
    result = pg.simulate(config, op=ops.BalaGoyalOp, size=config.network.size)
    results.append(result)
# Merge simulation results in a single data frame for post-processing.
# If stream is set, write the resulting data frame to a file
_ = metadata.merge(*results, stream='results.csv')
print('Bye.')

#### 1.2 Post-processing results

In [ ]:
import pandas as pd


df = pd.read_csv('results.csv')
df.head()

### 2 The effect of $\epsilon$



In [ ]:
# Enable auto-reloading
%reload_ext autoreload
# 2: Reload all
%autoreload 2


from collections import deque, namedtuple

import polygraphs as pg

from polygraphs import hyperparameters as hparams
from polygraphs import metadata
from polygraphs import ops


# Create base PolyGraph configuration. All simulations inherit from this configuration.
# In this particular case, we will vary the network size.
params = hparams.PolyGraphHyperParameters()
# Initial beliefs are random uniform between 0 and 1
params.init.kind = 'uniform'
# Chance that action B is better than action A
params.epsilon = 0.0
# Kind and size of network (a cycle network with 10 agents)
params.network.kind = 'cycle'
params.network.size = 10
# Disable logging
params.logging.enabled = False
# Repeat simulation 10 times; run to convergence
params.simulation.steps = 0
params.simulation.repeats = 100

# Generate a list of configurations, varying the network size
options = {'epsilon': [0.001, 0.025, 0.05, 0.075, 0.1]}
configurations = hparams.PolyGraphHyperParameters.expand(params, options)

results = deque()
for config in configurations:
    # Run experiment
    pg.log.info('Simulating a {}-node {} network of with epsilon {:5.3f}'.format(config.network.size,
                                                                                 config.network.kind,
                                                                                 config.epsilon))
    result = pg.simulate(config, op=ops.BalaGoyalOp, epsilon=config.epsilon)
    results.append(result)
# Merge simulation results in a single data frame for post-processing.
# If stream is set, write the resulting data frame to a file
_ = metadata.merge(*results, stream='epsilon.csv')
print('Bye.')

### 3 The effect of the number of trials

In [ ]:
# Enable auto-reloading
%reload_ext autoreload
# 2: Reload all
%autoreload 2


from collections import deque, namedtuple

import polygraphs as pg

from polygraphs import hyperparameters as hparams
from polygraphs import metadata
from polygraphs import ops


# Create base PolyGraph configuration. All simulations inherit from this configuration.
# In this particular case, we will vary the network size.
params = hparams.PolyGraphHyperParameters()
# Initial beliefs are random uniform between 0 and 1
params.init.kind = 'uniform'
# Chance that action B is better than action A
params.epsilon = 0.01
# Kind and size of network (a cycle network with 10 agents)
params.network.kind = 'cycle'
params.network.size = 10
# Disable logging
params.logging.enabled = False
# Repeat simulation 10 times; run to convergence
params.simulation.steps = 0
params.simulation.repeats = 100

# Generate a list of configurations, varying the network size
options = {'trials': [10, 100, 1000, 10000]}
configurations = hparams.PolyGraphHyperParameters.expand(params, options)

results = deque()
for config in configurations:
    # Run experiment
    pg.log.info('Simulating a {}-node {} network of with {:5d} trials'.format(config.network.size,
                                                                              config.network.kind,
                                                                              config.trials))
    result = pg.simulate(config, op=ops.BalaGoyalOp, trials=config.trials)
    results.append(result)
# Merge simulation results in a single data frame for post-processing.
# If stream is set, write the resulting data frame to a file
_ = metadata.merge(*results, stream='trials.csv')
print('Bye.')